<a href="https://colab.research.google.com/github/ankit-rathi/Quantvesting_v2/blob/main/myStocks_Portfolio_Analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install pyxirr
!pip install ta
!pip install yfinance==0.2.59

  Installing build dependencies ... -

 \

 done


  Getting requirements to build wheel ... -

 done


...

 done
  Created wheel for ta: filename=ta-0.11.0-py3-none-any.whl size=29482 sha256=e9867cf13ce3a52bed81f6af05fefacbde601ce994e167247aefdb4e071085a9
  Stored in directory: /home/runner/.cache/pip/wheels/ec/b6/81/d781ff040ee9432384560f29596ee14ca79d70529d4228e50f
Successfully built ta


  Attempting uninstall: protobuf
    Found existing installation: protobuf 6.33.2
    Uninstalling protobuf-6.33.2:
      Successfully uninstalled protobuf-6.33.2


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 0/2 [protobuf]

  Attempting uninstall: yfinance
    Found existing installation: yfinance 1.0
    Uninstalling yfinance-1.0:
      Successfully uninstalled yfinance-1.0
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 0/2 [protobuf]

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2/2 [yfinance]


In [2]:
# import libraries

import numpy as np
import pandas as pd
import datetime
import warnings
warnings.filterwarnings('ignore')
import yfinance as yfin
import ta
import matplotlib.pyplot as plt
import requests

In [3]:
# notebook start time
import time
import datetime
import pytz

start_time = time.time()

# function to print date timestamp
def print_date_time():
  tz_NY = pytz.timezone('Asia/Kolkata')
  datetime_NY = datetime.datetime.now(tz_NY)
  print("Run date time (IST):", datetime_NY.strftime("%Y-%m-%d %H:%M:%S"))

In [4]:
# function to get booked and reserved amount
def get_amt():
  # fetch the JSON data from the URL
  url = "https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myPortfolioAmts.json"
  response = requests.get(url)
  pf_amts = response.json()  # parse the JSON data

  # extract values from the JSON
  py_booked_amt_dm = pf_amts["py_booked_amt_dm"]
  py_booked_amt_sv = pf_amts["py_booked_amt_sv"]
  cy_booked_amt_dm = pf_amts["cy_booked_amt_dm"]
  cy_booked_amt_sv = pf_amts["cy_booked_amt_sv"]
  reserve_amt_dm = pf_amts["reserve_amt_dm"]
  reserve_amt_sv = pf_amts["reserve_amt_sv"]

  # perform calculations
  py_booked_amt = py_booked_amt_dm + py_booked_amt_sv
  cy_booked_amt = cy_booked_amt_dm + cy_booked_amt_sv
  reserve_amt = reserve_amt_dm + reserve_amt_sv
  total_booked_amt = py_booked_amt + cy_booked_amt

  return total_booked_amt, reserve_amt, cy_booked_amt, py_booked_amt

gen_output = 0

In [5]:
# function to format the amount
def format_amt(number):
    abs_number = abs(number)

    if abs_number >= 1_00_00_000:  # Crores
        formatted_amt = f"{number / 1_00_00_000:.2f} C"
    elif abs_number >= 1_00_000:  # Lakhs
        formatted_amt = f"{number / 1_00_000:.2f} L"
    elif abs_number >= 1_000:  # Thousands
        formatted_amt = f"{number / 1_000:.2f} K"
    else:
        formatted_amt = f"{number:.2f}"

    return formatted_amt

# function to set start and end date
def get_start_end_date():
  start_date = (datetime.date.today() + datetime.timedelta(days=-365)).strftime('%Y-%m-%d')
  end_date = (datetime.date.today() + datetime.timedelta(days=1)).strftime('%Y-%m-%d')
  return start_date, end_date

# function to fetch my portfolio csv
def get_mypfs_df():
  mypfs_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myPortfolioStocks.csv')
  return mypfs_df

# function to fetch my prospects csv
def get_mypps_df():
  mypps_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myProspectsScrips.csv')
  return mypps_df

# function to fetch screener data
def get_myscreen_df():
  myscreen_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myScreenerDB.csv')
  return myscreen_df

# function to fetch momentum data
def get_myinvmt_df():
  myinvmt_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myInvestments.csv')
  return myinvmt_df

# function to get the stock ids
def get_stock_ids(df_pf):
  stock_n100 = df_pf['Symbol'].unique()

  exclude = ['CADILAHC','MMTC', 'MASFIN']

  stock_ids = df_pf[~df_pf['Symbol'].isin(exclude) ]['Symbol'].unique()

  #mypf = mypf[mypf.Forecast.notnull()]
  #stock_ids = mypf['Symbol'].unique()

  stock_ids.sort()
  return stock_ids

# get features from screener data
def get_screener_features():
  myscreen_df = get_myscreen_df()
  cols = ['Symbol', 'EPS', 'MedPE', 'ROCE%', 'ROE%', 'CapType']
  return myscreen_df[cols]

# get relative strength
def get_relative_strength(stock_list):

    # Dictionary to store stock tickers and their corresponding percentage price change
    stock_changes = {}

    # Loop through each stock and fetch its price data
    for stock in stock_list:
        try:
            # Download the stock data for the given date range
            stock_data = yfin.Ticker(stock + '.NS').history(period='1mo', interval='1d')[map(str.title, ['open', 'close', 'low', 'high', 'volume'])]

            # Calculate the percentage change for the stock
            if len(stock_data) > 0:
                start_price = stock_data['Close'].iloc[0]
                end_price = stock_data['Close'].iloc[-1]
                percent_change = round(((end_price - start_price) / start_price) * 100, 2)
                stock_changes[stock] = percent_change
            else:
                stock_changes[stock] = np.nan  # If no data is available, set to NaN

        except Exception as e:
            print(f"Error fetching data for {stock}: {e}")
            stock_changes[stock] = np.nan

    # Create a DataFrame with stock tickers and their percentage changes
    df = pd.DataFrame(list(stock_changes.items()), columns=['Symbol', 'Percent_Change'])

    # Drop any stocks with missing data (NaN values)
    df = df.dropna()

    # Calculate the percentile rank based on percentage change
    df['RSP'] = round(df['Percent_Change'].rank(pct=True) * 100, 2)
    cols = ['Symbol', 'RSP']
    df = df[cols]
    # Sort by percentile rank (optional)
    df = df.sort_values(by='RSP', ascending=False).reset_index(drop=True)

    return df


In [6]:
# function to get stock technicals
def stock_prec_dev(stock_symbol):
    #stock_symbol = 'ULTRACEMCO.NS'
    short_window = 20
    mid_window = 50
    long_window = 200
    moving_avg = 'SMA'

    start = datetime.datetime(*map(int, start_date.split('-')))
    end = datetime.datetime(*map(int, end_date.split('-')))

    stock_df = yfin.Ticker(stock_symbol + '.NS').history(period='max', interval='1d')[map(str.title, ['open', 'close', 'low', 'high', 'volume'])]

    stock_df['Max'] = round(max(stock_df['Close']),2)
    stock_df = stock_df[(stock_df.index <= end_date) & (stock_df.index >= start_date)]
    stock_df['200_SMA'] = round(stock_df['Close'].rolling(window = 200, min_periods = 1).mean(),0)
    stock_df['Dev%_200'] = round((stock_df['Close'] - stock_df['200_SMA'])*100/stock_df['200_SMA'],2)
    stock_df.dropna(axis = 0, inplace = True) # remove any null rows

    stock_df['20_SMA'] = stock_df['Close'].rolling(window=20).mean()
    stock_df['50_SMA'] = stock_df['Close'].rolling(window=50).mean()
    stock_df['Symbol'] = stock_symbol

    stock_df['Close'] = round(stock_df['Close'],2)
    stock_df['Min'] = round(min(stock_df['Close']),2)
    stock_df['RSI_14'] = ta.momentum.RSIIndicator(close=stock_df['Close'], window=14).rsi()
    stock_df['RSI_14'] = round(stock_df['RSI_14'],0)
    stock_df['Prev_Close'] = stock_df['Close'].shift(1)
    stock_df.drop(['Open', 'Low', 'High', 'Volume'], axis=1, inplace=True)
    stock_df = stock_df.tail(1)

    max_SMA = max(stock_df['20_SMA'].item(), stock_df['50_SMA'].item(), stock_df['200_SMA'].item())
    min_SMA = min(stock_df['20_SMA'].item(), stock_df['50_SMA'].item(), stock_df['200_SMA'].item())
    ABS_Spread = max_SMA - min_SMA
    stock_df['Spread%'] = round((ABS_Spread / stock_df['200_SMA'].item()) * 100,2)

    return stock_df

# function to compute stock attributes
def get_common_features(stock_ids, df_mypf):

  df_prec_dev = pd.DataFrame()
  df_tmp = get_screener_features()
  df_rs = get_relative_strength(stock_ids)

  for stock_id in stock_ids:
      tmp = stock_prec_dev(stock_id)
      tmp = tmp.reset_index()
      df_prec_dev = pd.concat([df_prec_dev, tmp], ignore_index = True)
  df_prec_dev = pd.merge(df_prec_dev, df_mypf, on= 'Symbol')
  df_prec_dev = pd.merge(df_prec_dev, df_tmp, on= 'Symbol', how='left')
  df_prec_dev = pd.merge(df_prec_dev, df_rs, on= 'Symbol', how='left')
  #print(df_prec_dev.columns)
  df_prec_dev['Curr_PE'] = round(df_prec_dev['Close']/df_prec_dev['EPS'],1)
  df_prec_dev['Dev%_PE'] = round((df_prec_dev['Curr_PE'] - df_prec_dev['MedPE'])*100/df_prec_dev['MedPE'],2)
  df_prec_dev['Conviction'] = df_prec_dev['Conviction'] + '-' + df_prec_dev['CapType']
  return df_prec_dev

# function to arrange stock features
def arrange_features(df_stocks, common_cols, diff_cols):
  df_stocks_common = df_stocks[common_cols].drop_duplicates()
  df_stocks_diff = df_stocks[diff_cols]
  df_stocks_diff['Investment'] = df_stocks_diff['AvgCost'] * df_stocks_diff['Shares']
  df_stocks_diff = df_stocks_diff.groupby(['Symbol'])[['Shares', 'Investment']].aggregate(['sum']).reset_index()
  df_stocks_diff.columns = ['Symbol', 'Shares', 'Investment']
  df_stocks_diff['AvgCost'] = round(df_stocks_diff['Investment']/df_stocks_diff['Shares'],2)
  df_stocks = pd.merge(df_stocks_diff, df_stocks_common, on='Symbol')
  return df_stocks

# plot fact distribution across dimension
def plot_pie_chart(df, dimension, fact):
  # grouping the data by category and calculating the sum of fact for each type
  grouped_data = df.groupby(dimension)[fact].sum()

  # sorting the grouped data in descending order
  grouped_data = grouped_data.sort_values(ascending=False)

  # creating a pie chart
  grouped_data.plot.pie(autopct='%1.1f%%', startangle=90, figsize=(6, 6))

  # adding a title and displaying the plot
  plt.title(f'{dimension} {fact} Distribution')
  plt.ylabel('')  # To hide the y-label
  plt.show()

In [7]:
# function to get portfolio features

def get_portfolio_features(df_common_features):

  print_date_time()
  print('-------------------')

  #df_common_features["Target"] = df_common_features["Target"].fillna(df_common_features["Max"])
  df_common_features['NTT'] = np.where(df_common_features['Strategy']== 'NTT', df_common_features["Target"], df_common_features['Max'])
  df_common_features['LTT'] = np.where(df_common_features['Strategy']== 'BTT', df_common_features["Target"], df_common_features['Max'])
  df_common_features['BOL'] = df_common_features['Min']

  tmp_df = df_common_features[df_common_features['Symbol'].isin(mypf_df[mypf_df['InPortfolio'] != 'NA'].Symbol.values)]
  print('qualified stocks: '+str(len(tmp_df['Symbol'].unique())))
  tmp_df1 = tmp_df[tmp_df['LatestQtr'] == 1]
  print('with latest results: '+str(len(tmp_df1['Symbol'].unique())))
  tmp_df1 = tmp_df1[tmp_df1['StarStock'] == 1]
  print('still star stocks: '+str(len(tmp_df1['Symbol'].unique())))
  tmp_df['Investment'] = tmp_df['AvgCost'] * tmp_df['Shares']
  tmp_df['Current'] = round(tmp_df['Close'] * tmp_df['Shares'],0)
  tmp_df['Previous'] = tmp_df['Prev_Close'] * tmp_df['Shares']
  tmp_df['EstimatedST'] = tmp_df['NTT'] * tmp_df['Shares']
  tmp_df['EstimatedLT'] = tmp_df['LTT'] * tmp_df['Shares']
  tmp_df['Current P/L'] = round((tmp_df['Current'] - tmp_df['Investment']),0)
  tmp_df['Today P/L%'] = round((tmp_df['Current'] - tmp_df['Previous'])*100/tmp_df['Previous'],2)
  tmp_df['Current P/L%'] = round((tmp_df['Current'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['EstimatedST P/L%'] = round((tmp_df['EstimatedST'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['EstimatedLT P/L%'] = round((tmp_df['EstimatedLT'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['NTT%'] = round((tmp_df['NTT'] - tmp_df['Close'])*100/tmp_df['Close'],2)
  tmp_df['LTT%'] = round((tmp_df['LTT'] - tmp_df['Close'])*100/tmp_df['Close'],2)
  tmp_df['Gained%'] = round((tmp_df['Close'] - tmp_df['BOL'])*100/tmp_df['BOL'],2)
  investment = round(sum(tmp_df['AvgCost']*tmp_df['Shares']),0)
  current = round(sum(tmp_df['Close']*tmp_df['Shares']),0)
  tmp_df['InitAlloc%'] = round(tmp_df['Investment']*100/investment,2)
  tmp_df['CurrAlloc%'] = round(tmp_df['Current']*100/current,2)
  tmp_df['FTT'] = tmp_df['LTT']
  tmp_df.loc[tmp_df['Strategy'] == 'NTT', 'FTT'] = tmp_df['NTT']
  tmp_df['FTT%'] = tmp_df['LTT%']
  tmp_df.loc[tmp_df['Strategy'] == 'NTT', 'FTT%'] = tmp_df['NTT%']
  tmp_df['FTT Amt'] = round(tmp_df['FTT%'] * tmp_df['Current']/100,0)
  tmp_df['OTT%'] = round((tmp_df['FTT'] - tmp_df['AvgCost'])*100/tmp_df['AvgCost'],2)
  tmp_df['RRR Ind'] = round(tmp_df['Current P/L']/tmp_df['FTT Amt'],2)
  tmp_df['Risk Ind'] = round(tmp_df['Current P/L%']*tmp_df['CurrAlloc%'],0)

  return tmp_df

# function to print portfolio features
def print_portfolio_stats(df_portfolio_features, myinvmt_df):
  from pyxirr import xirr

  total_booked_amt, reserve_amt, cy_booked_amt, py_booked_amt = get_amt()

  dates = myinvmt_df['Date'].values
  dates = np.append(dates, datetime.date.today().strftime('%d-%b-%y'))
  investment = myinvmt_df['Investment'].values
  dates= pd.to_datetime(dates)

  current = round(sum(df_portfolio_features['Close']*df_portfolio_features['Shares']),0) + reserve_amt
  investment_xirr = np.append(investment, current)
  cagr = round(xirr(pd.DataFrame({"dates": dates, "amounts": investment_xirr}))*100,2)

  investment = -sum(investment)
  invested = round(sum(df_portfolio_features['AvgCost']*df_portfolio_features['Shares']),0) + reserve_amt
  previous = round(sum(df_portfolio_features['Prev_Close']*df_portfolio_features['Shares']),0) + reserve_amt
  cy_invested = investment + py_booked_amt

  today_pnl_amount = current-previous
  today_pnl_percentage = round((current-previous)*100/previous,2)

  curr_pnl_amount = current-invested
  curr_pnl_percentage = round((curr_pnl_amount)*100/(cy_invested),2)

  cy_pnl_amount = cy_booked_amt + curr_pnl_amount
  cy_pnl_percentage = round((cy_pnl_amount)*100/cy_invested,2)

  overall_pnl_amount = total_booked_amt + curr_pnl_amount
  overall_pnl_percentage = round((overall_pnl_amount)*100/investment,2)

  estimate_st = round(sum(df_portfolio_features['FTT']*df_portfolio_features['Shares']),0)  + reserve_amt
  est_st_pnl_amount = estimate_st-current
  est_st_pnl_percentage = round((est_st_pnl_amount)*100/current,2)

  estimate_lt = round(sum(df_portfolio_features['LTT']*df_portfolio_features['Shares']),0)  + reserve_amt
  est_lt_pnl_amount = estimate_lt-current
  est_lt_pnl_percentage = round((est_lt_pnl_amount)*100/current,2)

  total_profit = round(sum(df_portfolio_features[df_portfolio_features['Current P/L%'] > 0]['Current']) - sum(df_portfolio_features[df_portfolio_features['Current P/L%'] > 0]['Investment']),0)
  total_loss = round(sum(df_portfolio_features[df_portfolio_features['Current P/L%'] < 0]['Current']) - sum(df_portfolio_features[df_portfolio_features['Current P/L%'] < 0]['Investment']),0)

  cy_booked_percentage = round((cy_booked_amt)*100/current,2)
  py_booked_percentage = round((py_booked_amt)*100/investment,2)
  total_booked_percentage = round((total_booked_amt)*100/investment,2)

  print('-------------------')
  print('Initial Investment: ', format_amt(investment))
  print('CY Investment: ', format_amt(cy_invested))
  print('Reserve: ', format_amt(reserve_amt))
  print('Current: ',  format_amt(current))
  print('-------------------')
  print('Today PnL: '+ '{} ({}%)'.format(format_amt(today_pnl_amount), today_pnl_percentage))
  print('Current PnL: '+ '{} ({}%)'.format(format_amt(curr_pnl_amount), curr_pnl_percentage))
  print('CY Booked + Current PnL: '+ '{} ({}%)'.format(format_amt(cy_pnl_amount), cy_pnl_percentage))
  print('-------------------')
  print('Total profit: ', format_amt(total_profit))
  print('Total loss: ', format_amt(total_loss))
  print('-------------------')
  print('Total Booked + Current PnL: '+ '{} ({}%)'.format(format_amt(overall_pnl_amount), overall_pnl_percentage))
  print('Total Booked PnL: '+ '{} ({}%)'.format(format_amt(total_booked_amt), total_booked_percentage))
  print('Curr Year Booked PnL: '+ '{} ({}%)'.format(format_amt(cy_booked_amt), cy_booked_percentage))
  print('Prev Year Booked PnL: '+ '{} ({}%)'.format(format_amt(py_booked_amt), py_booked_percentage))
  print('===================')
  print('Est FTT: ',  format_amt(estimate_st))
  print('Est FTT PnL: '+ '{} ({}%)'.format(format_amt(est_st_pnl_amount), est_st_pnl_percentage))

  print('===================')
  print('Deployed: ', format_amt(investment))

  print('Current: ', format_amt(current))

  print('CAGR/XIRR %: '+'{}%'.format(cagr))

In [8]:
# get start and end date
start_date, end_date = get_start_end_date()

# get portfolio and prospects data
mypfs_df = get_mypfs_df()
mypps_df = get_mypps_df()
myinvmt_df = get_myinvmt_df()

# merge above datasets
mypf_df = pd.merge(mypfs_df, mypps_df, on="Symbol")

# seggregate dm and sv portfolio
dm_pf = mypf_df[mypf_df['InPortfolio'] == 'DM']
sv_pf = mypf_df[mypf_df['InPortfolio'] == 'SV']

# build portfolio stock dataframe
dm_stocks = get_stock_ids(dm_pf)
sv_stocks = get_stock_ids(sv_pf)
df_stocks = pd.concat([dm_pf,sv_pf], ignore_index = True)

# arrange common and diff stock features
common_cols = ['Symbol', 'Target', 'Criteria', 'Strategy', 'CumlRnk', 'LatestQtr', 'StarStock', 'Conviction', 'Category', 'InFolio', 'XIRR', 'MBQ']
diff_cols = ['Symbol', 'AvgCost', 'Shares']
df_stocks = arrange_features(df_stocks, common_cols, diff_cols)

# get common features
stock_ids = df_stocks['Symbol'].values
df_common_features = get_common_features(stock_ids, df_stocks)

df_common_features.reset_index(drop=True, inplace=True)
df_common_features.drop(['Date'], axis=1, inplace=True)
# get and print portfolio features
df_portfolio_features = get_portfolio_features(df_common_features)

print_portfolio_stats(df_portfolio_features, myinvmt_df)

df = df_portfolio_features
#plot_pie_chart(df, 'CapType', 'Current')
list_ox40n = list(df[df['MBQ'].str.contains('OX40N', na=False)]['Symbol'].values)

Run date time (IST): 2026-01-07 11:24:08
-------------------
qualified stocks: 86
with latest results: 25
still star stocks: 14
-------------------
Initial Investment:  1.30 C
CY Investment:  1.57 C
Reserve:  1.68 K
Current:  1.44 C
-------------------
Today PnL: 5.82 K (0.04%)
Current PnL: -24.69 L (-15.75%)
CY Booked + Current PnL: -10.40 L (-6.63%)
-------------------
Total profit:  2.21 L
Total loss:  -26.90 L
-------------------
Total Booked + Current PnL: 16.58 L (12.77%)
Total Booked PnL: 41.27 L (31.79%)
Curr Year Booked PnL: 14.29 L (9.9%)
Prev Year Booked PnL: 26.98 L (20.78%)
Est FTT:  2.34 C
Est FTT PnL: 89.66 L (62.1%)
Deployed:  1.30 C
Current:  1.44 C
CAGR/XIRR %: 5.83%


In [9]:
# top 5 near their targets
cols = ['Symbol', 'FTT','Today P/L%', 'Current P/L%', 'FTT%', 'OTT%', 'FTT Amt', 'Current P/L', 'Current', 'Dev%_PE', 'RSI_14','Conviction', 'Spread%',  'CumlRnk', 'RRR Ind', 'CurrAlloc%', 'Gained%', 'Criteria', 'Strategy', 'Category']
df_tmp = df_portfolio_features[~df_portfolio_features['Symbol'].isin(['ENRIN','BLUSPRING','DIGITIDE'])]
df_tmp = df_tmp.sort_values(by = 'FTT Amt', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
82,VOLTAS,1530.00,0.24,16.54,2.71,19.69,6056.0,31713.0,223455.0,5.14,71.0,X-MC,4.07,78.0,5.24,1.55,24.46,XY25,NTT,AC
51,NESTLEIND,1377.00,0.09,16.47,4.23,21.40,13076.0,43711.0,309137.0,8.10,71.0,X-LC,5.27,12.0,3.34,2.14,23.84,XY25,NTT,FMCG
49,MASFIN,398.61,0.01,3.54,17.88,22.05,18138.0,3465.0,101445.0,-10.57,64.0,H-SC,6.97,164.0,0.19,0.70,47.46,XR,ATH,FINANCE
77,TTKPRESTIG,770.00,-1.30,-20.71,26.23,0.09,20960.0,-20867.0,79910.0,85.49,38.0,M-SC,3.20,253.0,-1.00,0.55,4.98,OX40N,NTT,DURABLES
17,COALINDIA,484.83,-0.01,5.00,13.54,19.23,21392.0,7530.0,157990.0,22.81,82.0,L-LC,6.25,179.0,0.35,1.09,27.34,XY25,ATH,MINING


In [10]:
# top 5 today
df_tmp = df_portfolio_features.sort_values(by = 'Today P/L%', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
72,TATAELXSI,9161.00,4.13,-19.78,64.59,32.03,68307.0,-26081.0,105754.0,-13.18,65.0,H-SC,7.07,148.0,-0.38,0.73,18.45,OX40N,NTT,IT
13,BSOFT,831.70,3.43,-21.00,90.87,50.79,100971.0,-29537.0,111116.0,1.28,56.0,H-SC,10.56,151.0,-0.29,0.77,29.38,XR,ATH,IT
23,DMART,5391.45,3.18,-9.13,42.62,29.60,96670.0,-22782.0,226818.0,-22.53,46.0,X-LC,9.71,19.0,-0.24,1.57,11.06,X40N,ATH,FMCG
44,JCHAC,2282.00,2.71,-35.32,54.60,-0.01,45133.0,-45143.0,82662.0,16813.04,47.0,M-SC,14.15,233.0,-1.00,0.57,5.80,OX40N,BTT,AC
50,MEDANTA,1486.00,2.25,3.21,20.22,24.08,26494.0,4077.0,131027.0,-5.17,59.0,X-SC,6.72,89.0,0.15,0.91,23.46,XY24,NTT,HEALTHCARE


In [11]:
# bottom 5 today
df_tmp = df_portfolio_features.sort_values(by = 'Today P/L%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
56,RAJOOENG,143.1,-5.28,-28.20,94.43,39.61,69500.0,-28900.0,73600.0,-51.42,59.0,H-SC,31.69,135.0,-0.42,0.51,19.13,AR,ATH,MISC
75,TMPV,600.0,-2.07,-25.96,66.09,22.98,110541.0,-58639.0,167259.0,-23.08,48.0,X-LC,4.74,3.0,-0.53,1.16,5.20,XY24,NTT,AUTO
47,KANSAINER,340.0,-1.47,-21.00,43.64,13.47,92970.0,-56628.0,213039.0,-67.53,53.0,H-SC,5.05,158.0,-0.61,1.48,8.32,XY24,NTT,PAINTS
68,STARHEALTH,761.0,-1.45,-13.83,70.90,47.26,171750.0,-38880.0,242243.0,17.60,37.0,H-SC,8.17,171.0,-0.23,1.68,29.79,XY24,NTT,INSURANCE
40,INDUSINDBK,1800.0,-1.31,-33.39,99.49,32.89,51169.0,-25776.0,51431.0,-734.87,66.0,L-MC,7.49,259.0,-0.50,0.36,41.64,XR,NTT,BANKS


In [12]:
# top 5 to exit based on CumlRnk
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['Current P/L%'] > -1) & (df_tmp['Current P/L%'] < 1)].sort_values(by = 'CumlRnk', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
25,FINCABLES,1641.55,-0.01,-0.97,109.94,107.91,160749.0,-1429.0,146215.0,-19.34,56.0,M-SC,10.23,216.0,-0.01,1.01,9.02,OX40N,ATH,CABLES


In [13]:
# OX40N stocks to exit
df_tmp = df_portfolio_features[(df_portfolio_features['Symbol'].isin(list_ox40n))].sort_values(by = 'Current P/L%', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
25,FINCABLES,1641.55,-0.01,-0.97,109.94,107.91,160749.0,-1429.0,146215.0,-19.34,56.0,M-SC,10.23,216.0,-0.01,1.01,9.02,OX40N,ATH,CABLES
39,INDIGOPNTS,1408.00,1.28,-14.35,16.78,0.02,25087.0,-25052.0,149507.0,128.24,55.0,M-SC,7.49,234.0,-1.00,1.04,30.05,OX40N,NTT,PAINTS
72,TATAELXSI,9161.00,4.13,-19.78,64.59,32.03,68307.0,-26081.0,105754.0,-13.18,65.0,H-SC,7.07,148.0,-0.38,0.73,18.45,OX40N,NTT,IT
77,TTKPRESTIG,770.00,-1.30,-20.71,26.23,0.09,20960.0,-20867.0,79910.0,85.49,38.0,M-SC,3.20,253.0,-1.00,0.55,4.98,OX40N,NTT,DURABLES
47,KANSAINER,340.00,-1.47,-21.00,43.64,13.47,92970.0,-56628.0,213039.0,-67.53,53.0,H-SC,5.05,158.0,-0.61,1.48,8.32,XY24,NTT,PAINTS


In [14]:
# top 5 to book for rotation
df_tmp = df_portfolio_features[(df_portfolio_features['Current P/L%'] > 20) ].sort_values(by = 'CumlRnk', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category


In [15]:
# top 5 to monitor for exit
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['Current P/L%'] > 0) ].sort_values(by = 'Dev%_PE', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
57,RECLTD,446.00,0.40,4.06,16.02,20.74,33869.0,8250.0,211420.0,38.17,74.0,M-MC,6.30,189.0,0.24,1.46,15.11,XY25,NTT,FINANCE
17,COALINDIA,484.83,-0.01,5.00,13.54,19.23,21392.0,7530.0,157990.0,22.81,82.0,L-LC,6.25,179.0,0.35,1.09,27.34,XY25,ATH,MINING
84,WIPRO,420.00,1.85,10.95,55.27,72.27,103906.0,18550.0,187998.0,-4.72,63.0,M-LC,6.07,99.0,0.18,1.30,17.80,XR,NTT,IT
49,MASFIN,398.61,0.01,3.54,17.88,22.05,18138.0,3465.0,101445.0,-10.57,64.0,H-SC,6.97,164.0,0.19,0.70,47.46,XR,ATH,FINANCE
37,IEX,219.00,1.55,2.46,45.05,48.63,92298.0,4926.0,204880.0,-31.74,68.0,H-SC,13.95,136.0,0.05,1.42,14.37,XR,NTT,MISC


In [16]:
# top 5 to book for rotation from weak conviction
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[~df_tmp['Criteria'].isin(['XY25','XY24','X40', 'X40N'])].sort_values(by = 'RRR Ind', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
49,MASFIN,398.61,0.01,3.54,17.88,22.05,18138.0,3465.0,101445.0,-10.57,64.0,H-SC,6.97,164.0,0.19,0.70,47.46,XR,ATH,FINANCE
84,WIPRO,420.00,1.85,10.95,55.27,72.27,103906.0,18550.0,187998.0,-4.72,63.0,M-LC,6.07,99.0,0.18,1.30,17.80,XR,NTT,IT
37,IEX,219.00,1.55,2.46,45.05,48.63,92298.0,4926.0,204880.0,-31.74,68.0,H-SC,13.95,136.0,0.05,1.42,14.37,XR,NTT,MISC
25,FINCABLES,1641.55,-0.01,-0.97,109.94,107.91,160749.0,-1429.0,146215.0,-19.34,56.0,M-SC,10.23,216.0,-0.01,1.01,9.02,OX40N,ATH,CABLES
85,ZYDUSLIFE,1286.17,0.20,-1.62,37.86,35.63,78061.0,-3392.0,206182.0,-15.41,56.0,H-MC,2.90,119.0,-0.04,1.43,14.78,AR,ATH,PHARMA


In [17]:
# top 5 to accumulate based on lowest RSI_14
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])].sort_values(by = 'RSI_14', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
43,ITC,452.00,-0.18,-16.22,32.22,10.78,64875.0,-38968.0,201350.0,-50.14,13.0,X-LC,5.00,5.0,-0.60,1.39,0.00,X40,NTT,FMCG
7,AWL,485.00,-0.25,-25.38,110.66,57.19,269294.0,-82774.0,243353.0,-64.78,29.0,X-MC,8.42,58.0,-0.31,1.69,0.00,XY24,NTT,FMCG
22,DIXON,18931.72,0.77,-18.08,60.22,31.26,113850.0,-41712.0,189056.0,-61.77,30.0,X-MC,18.16,56.0,-0.37,1.31,0.77,X40N,ATH,IT
52,PAGEIND,51605.07,-0.17,-10.65,46.19,30.62,97830.0,-25254.0,211800.0,-36.47,32.0,X-MC,16.02,55.0,-0.26,1.47,0.00,X40,ATH,APPARELS
11,BATAINDIA,2096.00,-0.95,-39.56,123.70,35.20,96198.0,-50903.0,77767.0,1.95,36.0,X-SC,17.25,92.0,-0.53,0.54,0.00,X40,NTT,FOOTWEAR


In [18]:
# top 5 to accumulate based on Spread and CumlRnk
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['CumlRnk'] < 100)].sort_values(by = 'Spread%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
78,UNITDSPR,1644.00,-0.54,0.57,20.08,20.77,47285.0,1341.0,235485.0,-8.21,37.0,X-MC,0.97,62.0,0.03,1.63,7.53,X40N,NTT,BREWERIES
65,SIEMENS,4671.50,-0.76,-16.60,50.49,25.51,78366.0,-30885.0,155210.0,-4.77,47.0,H-LC,1.43,49.0,-0.39,1.08,14.38,AR,ATH,ELECTRICAL
20,DABUR,735.00,0.06,1.79,41.17,43.70,103960.0,4447.0,252515.0,-6.68,66.0,X-MC,2.01,72.0,0.04,1.75,17.11,XY24,BTT,FMCG
81,VBL,671.64,0.58,1.35,33.86,35.67,108391.0,4274.0,320116.0,-10.77,67.0,X-LC,2.13,4.0,0.04,2.22,15.40,X40N,ATH,FMCG
45,JIOFIN,387.00,-0.62,-3.97,29.63,24.48,76076.0,-10612.0,256753.0,-5.55,50.0,X-LC,2.30,36.0,-0.14,1.78,48.82,XY24,BTT,FINANCE


In [19]:
# top 5 to accumulate based on lowest Gained%
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])].sort_values(by = 'Gained%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
7,AWL,485.00,-0.25,-25.38,110.66,57.19,269294.0,-82774.0,243353.0,-64.78,29.0,X-MC,8.42,58.0,-0.31,1.69,0.00,XY24,NTT,FMCG
11,BATAINDIA,2096.00,-0.95,-39.56,123.70,35.20,96198.0,-50903.0,77767.0,1.95,36.0,X-SC,17.25,92.0,-0.53,0.54,0.00,X40,NTT,FOOTWEAR
52,PAGEIND,51605.07,-0.17,-10.65,46.19,30.62,97830.0,-25254.0,211800.0,-36.47,32.0,X-MC,16.02,55.0,-0.26,1.47,0.00,X40,ATH,APPARELS
43,ITC,452.00,-0.18,-16.22,32.22,10.78,64875.0,-38968.0,201350.0,-50.14,13.0,X-LC,5.00,5.0,-0.60,1.39,0.00,X40,NTT,FMCG
22,DIXON,18931.72,0.77,-18.08,60.22,31.26,113850.0,-41712.0,189056.0,-61.77,30.0,X-MC,18.16,56.0,-0.37,1.31,0.77,X40N,ATH,IT


In [20]:
# top 5 to accumulate based on lowest CurrAlloc%
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])].sort_values(by = 'CurrAlloc%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
54,QUESS,424.00,1.20,-25.44,91.58,42.84,44388.0,-16537.0,48469.0,-50.96,60.0,X-SC,28.50,83.0,-0.37,0.34,9.66,XY24,NTT,MISC
58,RELAXO,1176.00,-0.63,-47.96,192.68,52.31,145547.0,-69622.0,75538.0,-44.63,43.0,X-SC,7.60,91.0,-0.48,0.52,1.11,X40N,NTT,FOOTWEAR
11,BATAINDIA,2096.00,-0.95,-39.56,123.70,35.20,96198.0,-50903.0,77767.0,1.95,36.0,X-SC,17.25,92.0,-0.53,0.54,0.00,X40,NTT,FOOTWEAR
50,MEDANTA,1486.00,2.25,3.21,20.22,24.08,26494.0,4077.0,131027.0,-5.17,59.0,X-SC,6.72,89.0,0.15,0.91,23.46,XY24,NTT,HEALTHCARE
34,HONAUT,58357.33,-0.35,-16.36,71.36,43.32,97207.0,-26652.0,136220.0,-28.18,51.0,X-SC,7.78,90.0,-0.27,0.94,4.59,X40N,ATH,ELECTRICAL


In [21]:
# top 5 to accumulate based on CumlRnk
df_tmp = df_portfolio_features.sort_values(by = 'CumlRnk', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
73,TCS,4389.97,0.76,-9.80,33.82,20.71,115381.0,-37075.0,341162.0,-22.33,58.0,X-LC,2.66,1.0,-0.32,2.36,14.0,X40,ATH,IT
41,INFY,1972.00,1.14,11.00,20.94,34.24,72728.0,34430.0,347318.0,-13.50,55.0,X-LC,7.54,2.0,0.47,2.41,20.2,X40,NTT,IT
75,TMPV,600.00,-2.07,-25.96,66.09,22.98,110541.0,-58639.0,167259.0,-23.08,48.0,X-LC,4.74,3.0,-0.53,1.16,5.2,XY24,NTT,AUTO
81,VBL,671.64,0.58,1.35,33.86,35.67,108391.0,4274.0,320116.0,-10.77,67.0,X-LC,2.13,4.0,0.04,2.22,15.4,X40N,ATH,FMCG
43,ITC,452.00,-0.18,-16.22,32.22,10.78,64875.0,-38968.0,201350.0,-50.14,13.0,X-LC,5.00,5.0,-0.60,1.39,0.0,X40,NTT,FMCG


In [22]:
# top 5 for average up
df_tmp = df_portfolio_features[(df_portfolio_features['Dev%_200'] > 5)].sort_values(by = 'CurrAlloc%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
40,INDUSINDBK,1800.00,-1.31,-33.39,99.49,32.89,51169.0,-25776.0,51431.0,-734.87,66.0,L-MC,7.49,259.0,-0.50,0.36,41.64,XR,NTT,BANKS
49,MASFIN,398.61,0.01,3.54,17.88,22.05,18138.0,3465.0,101445.0,-10.57,64.0,H-SC,6.97,164.0,0.19,0.70,47.46,XR,ATH,FINANCE
13,BSOFT,831.70,3.43,-21.00,90.87,50.79,100971.0,-29537.0,111116.0,1.28,56.0,H-SC,10.56,151.0,-0.29,0.77,29.38,XR,ATH,IT
69,SURYODAY,216.00,-1.22,-18.81,51.55,23.03,74944.0,-33690.0,145381.0,57.89,54.0,H-SC,5.19,167.0,-0.45,1.01,44.25,XR,NTT,BANKS
39,INDIGOPNTS,1408.00,1.28,-14.35,16.78,0.02,25087.0,-25052.0,149507.0,128.24,55.0,M-SC,7.49,234.0,-1.00,1.04,30.05,OX40N,NTT,PAINTS


In [23]:
# top 5 RSP
df_tmp = df_portfolio_features.sort_values(by = 'RSP', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
0,5PAISA,593.00,-1.17,-27.00,54.67,12.90,87195.0,-59002.0,159494.0,135.58,70.0,H-SC,10.60,161.0,-0.68,1.10,32.62,OX40N,NTT,FINANCE
17,COALINDIA,484.83,-0.01,5.00,13.54,19.23,21392.0,7530.0,157990.0,22.81,82.0,L-LC,6.25,179.0,0.35,1.09,27.34,XY25,ATH,MINING
82,VOLTAS,1530.00,0.24,16.54,2.71,19.69,6056.0,31713.0,223455.0,5.14,71.0,X-MC,4.07,78.0,5.24,1.55,24.46,XY25,NTT,AC
57,RECLTD,446.00,0.40,4.06,16.02,20.74,33869.0,8250.0,211420.0,38.17,74.0,M-MC,6.30,189.0,0.24,1.46,15.11,XY25,NTT,FINANCE
18,COFFEEDAY,80.00,2.21,-39.72,124.40,35.27,85149.0,-45101.0,68448.0,-53.72,50.0,L-SC,3.06,270.0,-0.53,0.47,67.53,XR,NTT,HOTELS


In [24]:
# Top N allocation
df_tmp = df_portfolio_features.sort_values(by = 'CurrAlloc%', ascending=False)
top_n_values = [10, 25, 50]

sum_df = pd.DataFrame({
    'Top_N': top_n_values,
    'Sum_Alloc%': [df_tmp['CurrAlloc%'].head(n).sum() for n in top_n_values]
})

sum_df

,Top_N,Sum_Alloc%
0,10,20.18
1,25,44.18
2,50,76.44


In [25]:
# market-cap-wise allocation
df_tmp = df_portfolio_features[cols]
df_tmp.groupby(df_tmp['Conviction'].str[-2:])['CurrAlloc%'].sum().sort_values(ascending=False)

Conviction
SC    42.89
MC    30.81
LC    26.34
Name: CurrAlloc%, dtype: float64

In [26]:
# criteria-wise allocation
df_portfolio_features.groupby('Criteria')['CurrAlloc%'].sum().sort_values(ascending=False)

Criteria
XY24     26.44
X40      23.85
X40N     12.74
XY25     10.69
XR        9.12
AR        8.65
OX40N     7.59
SR        0.96
Name: CurrAlloc%, dtype: float64

In [27]:
# conviction-wise allocation
df_portfolio_features.groupby('Conviction')['CurrAlloc%'].sum().sort_values(ascending=False)

Conviction
X-MC    24.30
H-SC    22.95
X-LC    22.87
M-SC    11.27
X-SC     7.88
H-MC     4.69
M-MC     1.46
M-LC     1.30
L-LC     1.09
H-LC     1.08
L-SC     0.79
L-MC     0.36
Name: CurrAlloc%, dtype: float64

In [28]:
# sector-wise stats
df_tmp = df_portfolio_features.groupby('Category')[['CurrAlloc%', 'Current', 'Current P/L', 'FTT Amt']].sum().sort_values(by=['Current', 'Current P/L'], ascending=False)
df_tmp['Current P/L%'] = round(df_tmp['Current P/L'] * 100 / df_tmp['Current'], 2)
df_tmp['FTT%'] = round(df_tmp['FTT Amt'] * 100 / df_tmp['Current'], 2)
cols = ['CurrAlloc%', 'Current P/L%', 'FTT%']
df_tmp[cols].sort_values(by=['CurrAlloc%'], ascending=False)

,CurrAlloc%,Current P/L%,FTT%
Category,,,
FMCG,16.97,-7.53,42.81
IT,12.78,-15.94,74.42
FINANCE,11.15,-10.66,57.10
MISC,7.20,-25.19,76.40
ELECTRICAL,5.93,-9.81,50.03
PAINTS,5.72,-12.11,28.76
INSURANCE,4.69,-0.09,34.34
PHARMA,3.99,-2.07,34.20
AUTO,2.80,-29.41,74.27


In [29]:
# money to be made criteria-wise
df_portfolio_features.groupby('Criteria')['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

,sum,count
Criteria,,
XY24,3157176.0,21
AR,1324972.0,10
XR,1175645.0,12
X40,1079278.0,15
X40N,901910.0,9
OX40N,694826.0,10
XY25,349448.0,7
SR,282873.0,2


In [30]:
# money to be made conviction-wise
df_portfolio_features.groupby('Conviction')['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

,sum,count
Conviction,,
H-SC,3483136.0,24
X-MC,1479262.0,16
M-SC,1430455.0,15
X-LC,923317.0,13
X-SC,782260.0,8
H-MC,403149.0,3
L-SC,175847.0,2
M-LC,103906.0,1
H-LC,78366.0,1


In [31]:
# money to be made criteria and conviction-wise
df_portfolio_features.groupby(['Conviction', 'Criteria'])['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

sum  count
Conviction Criteria                  
H-SC       XY24      1231731.0      6
           AR         910824.0      5
M-SC       XY24       821159.0      6
H-SC       XR         754295.0      7
X-MC       X40        481554.0      7
           XY24       436989.0      3
X-LC       X40        409003.0      6
X-MC       X40N       389168.0      4
X-SC       X40N       307681.0      3
H-SC       OX40N      303413.0      4
M-SC       OX40N      300715.0      5
X-SC       XY24       285858.0      3
H-SC       SR         282873.0      2
H-MC       AR         208327.0      2
X-LC       X40N       205061.0      2
H-MC       XY24       194822.0      1
X-SC       X40        188721.0      2
X-LC       XY24       186617.0      2
M-SC       XR         181126.0      2
X-MC       XY25       171551.0      2
M-SC       AR         127455.0      2
X-LC       XY25       122636.0      3
M-LC       XR         103906.0      1
L-SC       OX40N       90698.0      1
           XR          85149.0      1
H-LC       AR          78366.0      1
L-MC       XR          51169.0      1
M-MC       XY25        33869.0      1
L-LC       XY25        21392.0      1

In [32]:
# notebook execution time

end_time = time.time()
execution_time = round(end_time - start_time, 0)
print(f"Notebook execution time: {execution_time} seconds")

Notebook execution time: 39.0 seconds
